# Finetune (training) Hands-on

<https://huggingface.co/docs/transformers/ja/training>

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

## Preparing dataset

データセットを読み込む。データセットは [YelpReviewFull](https://huggingface.co/datasets/Yelp/yelp_review_full) でローカルビジネスのレビューとその点数。

In [2]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

トークナイザ(BERTより)を準備

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

データセットをあらかじめトークナイズしちゃう

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

実行時間を短くするために、サブセットで学習&評価する

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Train with Pytorch Trainer

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

モデル: Google BERT を読み込む。FP32でGPUへ読み込まれる。

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

for name, param in model.named_parameters():
    print(f"{name:80s} {str(param.shape):30s} {param.dtype} {param.is_cuda}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.embeddings.word_embeddings.weight                                           torch.Size([28996, 768])       torch.float32 False
bert.embeddings.position_embeddings.weight                                       torch.Size([512, 768])         torch.float32 False
bert.embeddings.token_type_embeddings.weight                                     torch.Size([2, 768])           torch.float32 False
bert.embeddings.LayerNorm.weight                                                 torch.Size([768])              torch.float32 False
bert.embeddings.LayerNorm.bias                                                   torch.Size([768])              torch.float32 False
bert.encoder.layer.0.attention.self.query.weight                                 torch.Size([768, 768])         torch.float32 False
bert.encoder.layer.0.attention.self.query.bias                                   torch.Size([768])              torch.float32 False
bert.encoder.layer.0.attention.self.key.weight                              

メトリクスの計算関数を定義。メトリクスには [acculacy](https://huggingface.co/spaces/evaluate-metric/accuracy) を利用する。

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print("logits", logits[0:10])
    predictions = np.argmax(logits, axis=-1)
    print("predictions:", predictions[0:10])
    print("references:", labels[0:10])
    return metric.compute(predictions=predictions, references=labels)

Trainerの引数を作成
* 結果を `test_trainer` ディレクトリに確認
* `epoch` ベース(学習の進行がある程度進んだら)で、評価を実行

In [10]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

学習≒ファインチューニング実行

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
%%time
trainer.train()

D:\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.186469,0.524000
2,No log,1.116050,0.521000
3,No log,1.009289,0.567000


logits [[-0.93449664 -0.9713814  -0.60252225  0.8732902   0.98570895]
 [-1.5720049  -0.87890637  0.5544452   1.4727589   0.21361467]
 [ 0.49420854  0.41460916  0.01131687 -0.63021266 -1.0580003 ]
 [-1.4877151  -1.0188559   0.08496182  1.3169426   0.52965707]
 [-1.3845254  -0.62241495  0.71626604  1.342631   -0.07601081]
 [-0.48736817 -0.94748807 -1.0185044   0.559435    1.0794078 ]
 [ 0.38069475  0.6131936   0.2743285  -0.35604873 -1.3995923 ]
 [-1.3487667  -0.99230945  0.14259902  1.2253498   0.4714005 ]
 [-1.3784069  -0.6757421   0.33992442  1.2012279   0.17523302]
 [-1.3365096  -0.601864    0.3648278   1.2511451  -0.03756431]]
predictions: [4 3 0 3 3 4 1 3 3 3]
references: [2 4 1 4 3 4 2 3 2 3]
logits [[-1.5863048  -1.4078939  -0.84005666  1.1638777   1.8756113 ]
 [-2.659988   -0.922395    0.8436172   1.7458456   0.898182  ]
 [ 2.8785102   0.3616666  -0.6105196  -1.5433638  -1.2276707 ]
 [-2.099514   -1.3899739  -0.31696904  1.6079485   1.6714926 ]
 [-2.2842567  -1.2317963  -0.00803

TrainOutput(global_step=375, training_loss=1.1024170735677083, metrics={'train_runtime': 108.765, 'train_samples_per_second': 27.582, 'train_steps_per_second': 3.448, 'total_flos': 789354427392000.0, 'train_loss': 1.1024170735677083, 'epoch': 3.0})